<a href="https://colab.research.google.com/github/DU-ds/pyspark_udemy/blob/main/Section4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! apt update
! apt install openjdk-8-jdk-headless -qq > /dev/null
! wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
! tar xf spark-2.3.1-bin-hadoop2.7.tgz
! pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

! ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

from pyspark.sql import types




Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [5]:
df = spark.read.csv("challenge.csv", header=True)
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [10]:
from pyspark.sql import functions as sqlf
df_mex = df.withColumn("mexico", sqlf.when(df.Country == "Mexico", "YES").otherwise("NO"))
df_mex.show()

+---------------+--------------+-----------------+----------+------+
|     ip_address|       Country|      Domain Name|Bytes_used|mexico|
+---------------+--------------+-----------------+----------+------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|    NO|
| 119.239.207.13|         China|         youtu.be|        51|    NO|
|  68.69.217.210|         China|        adobe.com|        10|    NO|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|    NO|
|   211.13.10.68|     Indonesia|          hud.gov|        29|    NO|
|   239.80.21.97|      Suriname|       smh.com.au|       218|    NO|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|    NO|
| 127.242.24.138|         China| surveymonkey.com|       123|    NO|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|    NO|
|   237.54.11.63|         China|       amazon.com|        83|    NO|
| 252.141.157.25|         Japan|      cornell.edu|       374|    NO|
|185.220.128.248|       Belgium|  

In [11]:
df_mex.groupBy("mexico").agg(sqlf.sum(df.Bytes_used)).alias("Bytes_Mexico").show()

+------+---------------+
|mexico|sum(Bytes_used)|
+------+---------------+
|   YES|         6293.0|
|    NO|       508076.0|
+------+---------------+



In [18]:
df_ip_country = df.groupBy("Country").agg(sqlf.countDistinct("ip_address").alias("ips_per_country"))
df_ip_country.sort(col("ips_per_country").desc()).show()

+--------------+---------------+
|       Country|ips_per_country|
+--------------+---------------+
|         China|            172|
|     Indonesia|            114|
|   Philippines|             65|
|        Russia|             56|
|        Brazil|             35|
|        Poland|             31|
|        Sweden|             28|
|         Japan|             25|
|      Portugal|             23|
|Czech Republic|             23|
|        France|             21|
|          Peru|             19|
|      Colombia|             17|
| United States|             15|
|     Argentina|             14|
|       Ukraine|             14|
|        Mexico|             13|
|      Thailand|             12|
|       Nigeria|             11|
|        Canada|             11|
+--------------+---------------+
only showing top 20 rows

